### LSE Data Analytics Online Career Accelerator 
# Course 301: Advanced Analytics for Organisational Impact

## Practical activity: Summarising model outputs

**This is the solution to the activity.**

Your client, age 45, wants to retire when their wealth exceeds £500,000. The current salary is £35,000, the cost of living is 6%, the savings rate is 6%, an interest rate of 2%, and they recieve promotions every year at a promotion raise of 6%. Create and test a model that will allow us to calculate how many years until the client's preferred level of savings is reached. 

Will your client be able to retire at age 65? To prepare your findings for presentation, you now need to summarise your results and visualise the change in wealth over time for the client. 

Create and test a model that will allow you to calculate how many years until the client's preferred level of savings is reached. Will 20 years be enough time? 

## 1. Prepare your workstation

In [ ]:
# Import the necessary libraries.
import pandas as pd
import random
import matplotlib.pyplot as plt
# Optional
%matplotlib inline

from dataclasses import dataclass
from sensitivity import SensitivityAnalyzer

## 2. Specify the model inputs

In [ ]:
# Create a class and instance.
@dataclass
class ModelInputs:
    starting_salary: int = 35000
    promos_every_n_years: int = 1
    cost_of_living_raise: float = 0.06
    promo_raise: float = 0.06
    savings_rate: float = 0.06
    interest_rate: float = 0.02
    desired_cash: int = 500000
        
model_data = ModelInputs()

model_data

## 3. Perform calculations

In [ ]:
# Calculate wage at a given year from start of the model based on cost of living, 
# raises, and regular promotion.
def wages_year(data: ModelInputs, year)->float:
    """
    Gets the wage at a given year from the start of the model based 
    on cost of living raises and regular promotions.
    """
    # Every n years we have a promotion, so dividing the years and taking 
    # out the decimals gets the number of promotions.
    num_promos = int(year / data.promos_every_n_years)
    
    # This is the formula above implemented in Python.
    salary_t = data.starting_salary * (1 + data.cost_of_living_raise)\
    ** year * (1 + data.promo_raise) ** num_promos
    return salary_t

for i in range(4):
    year = i + 1
    salary = wages_year(model_data, year)
    print(f"The wage at year {year} is £{salary:,.0f}.")

In [ ]:
def cash_saved_during_year(data: ModelInputs, year):
    """
    Calculated the cash saved within a given year, by first 
    calculating the salary at that year then applying the 
    savings rate.
    """
    salary = wages_year(data, year)
    cash_saved = salary * data.savings_rate
    return cash_saved

In [ ]:
def wealth_year(data: ModelInputs, year, prior_wealth):
    """
    Calculate the accumulated wealth for a given year, 
    based on previous wealth, the investment rate,
    and cash saved during the year.
    """
    cash_saved = cash_saved_during_year(data, year)
    wealth = prior_wealth * (1 + data.interest_rate) + cash_saved
    return wealth

In [ ]:
# Starting with no cash saved.
prior_wealth = 0  
for i in range(4):
    year = i + 1
    wealth = wealth_year(model_data, year, prior_wealth)
    print(f"The wealth at year {year} is £{wealth:,.0f}.")
    
    # Set next year's prior wealth to this year's wealth.
    prior_wealth = wealth

In [ ]:
# Runs the retirement model; collects salary and wealth information year by year
# and stores it in a DataFrame for further analysis.
def get_salaries_wealths_df(data):
    # Start with no cash saved.
    prior_wealth = 0   
    wealth = 0
    # The ‘year’ becomes ‘1’ on the first loop.
    year = 0  
    
    df_data_tups = []
    while wealth < data.desired_cash:
        year = year + 1
        salary = wages_year(data, year)
        wealth = wealth_year(data, year, prior_wealth)

        # Set next year's prior wealth to this year's wealth.
        prior_wealth = wealth 

        # Save the results in a tuple for later building the DataFrame.
        df_data_tups.append((year, salary, wealth))
        
    # Now we have run the while loop, so wealth must be >= desired_cash. 
    # Create the DataFrame.
    df = pd.DataFrame(df_data_tups, columns=['Year', 'Salary', 'Wealth'])
    
    return df

In [ ]:
# Save the function results into the DataFrame.
results_df = get_salaries_wealths_df(model_data)

# View the DataFrame.
results_df.head(10)

## 4. Create visualisation

In [ ]:
# Visualise wage or salary change over time.
results_df.plot('Year', 'Salary')

In [ ]:
# Visualise wealth change over time.
results_df.plot('Year', 'Wealth')

## Alternative: Years until retirement

In [ ]:
def years_to_retirement(data: ModelInputs, print_output=True):
    # Start with no cash saved.
    prior_wealth = 0  
    wealth = 0
    # The ‘year’ becomes ‘1’ on the first loop.
    year = 0  
   
    if print_output:
        print('Wealths over time:')
    while wealth < data.desired_cash:
        year = year + 1
        wealth = wealth_year(data, year, prior_wealth)
        if print_output:
            print(f'The accumulated wealth at year {year} is £{wealth:,.0f}.')
            # Set next year's prior wealth to this year's wealth.
        prior_wealth = wealth  
       
    # Now we have run the while loop, the wealth must be >= desired_cash 
    # (whatever last year was set is the years to retirement), which we can print.
    if print_output:
        # \n makes a blank line in the output.
        print(f'\nRetirement:\nIt will take {year} years to retire.')  
    return year

In [ ]:
years = years_to_retirement(model_data)